In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/My\ Drive

/content/gdrive/My Drive


In [ ]:
df = pd.read_excel('review_table.xlsx')

In [ ]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,BYOBCorkage,DriveThru,review_id,user_id,stars_y,useful,funny,cool,text,date
0,cxupofYLq46bxBHpKyFwsQ,Wyndham New Orleans - French Quarter,124 Royal Street,New Orleans,LA,70130,29.953713,-90.068526,3.0,266,...,NaN,NaN,xlK3ZAQrdy3xE7IpRL_Ucg,UG7f60WHPvWLUcGPIbm2ZA,2,2,1,1,"This hotel is bad, borderline mediocre. Of cou...",2014-08-24 19:33:12
1,cxupofYLq46bxBHpKyFwsQ,Wyndham New Orleans - French Quarter,124 Royal Street,New Orleans,LA,70130,29.953713,-90.068526,3.0,266,...,NaN,NaN,_dNI9YgOU9S0CCK16taQzQ,LHUrjf1H_ZLoqriutWEAeA,1,1,0,0,Had booked rooms a month in advance for a bach...,2014-03-14 20:25:53
2,hkh5iyDeGLNhu-JUQFRE1Q,Royal Sonesta New Orleans,300 Bourbon St,New Orleans,LA,70130,29.955761,-90.067887,3.5,492,...,NaN,NaN,Yn_RJRSvv0K6Ewh0Dy_9uQ,QZNEWt07tOTKh86oBUSwnA,2,6,0,1,"This hotel seems very nice in the lobby, but t...",2012-09-10 18:42:46
3,hkh5iyDeGLNhu-JUQFRE1Q,Royal Sonesta New Orleans,300 Bourbon St,New Orleans,LA,70130,29.955761,-90.067887,3.5,492,...,NaN,NaN,t6qCL6n8VOMLQIYy1GgG8g,0cH1enc41HLNuBFizzDEsw,5,2,0,0,Understand first that this was a party trip wi...,2012-12-05 19:22:23
4,hkh5iyDeGLNhu-JUQFRE1Q,Royal Sonesta New Orleans,300 Bourbon St,New Orleans,LA,70130,29.955761,-90.067887,3.5,492,...,NaN,NaN,3J0DcFukAxpmR9JXByAxFA,Gb8xOaeWE1cLMRZUHWfvKQ,5,3,1,0,A group of 12 girls and I came here for a Bach...,2014-09-25 15:57:00


In [ ]:
pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2) # binary classification: positive or negative sentiment

# Define a custom dataset class for processing the text data
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt',
            truncation=True
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten()
        }

# Define function for sentiment analysis
def analyze_sentiment(texts, model, tokenizer, max_len=128, batch_size=8):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    dataset = TextDataset(texts, tokenizer, max_len)
    dataloader = DataLoader(dataset, batch_size=batch_size)

    predictions = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Analyzing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            _, predicted_class = torch.max(probabilities, dim=1)
            predictions.extend(predicted_class.cpu().numpy())

    return predictions

# Sample text data
texts = df['text'].tolist()

# Perform sentiment analysis
predictions = analyze_sentiment(texts, model, tokenizer)

# Map predictions to sentiment labels
sentiment_labels = ['Negative' if pred == 0 else 'Positive' for pred in predictions]

# Add sentiment labels to the dataframe
df['sentiment'] = sentiment_labels

# Print the dataframe with sentiment labels
print(df[['text', 'sentiment']])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Analyzing: 100%|██████████| 2026/2026 [1:42:02<00:00,  3.02s/it]

                                                    text sentiment
0      This hotel is bad, borderline mediocre. Of cou...  Negative
1      Had booked rooms a month in advance for a bach...  Negative
2      This hotel seems very nice in the lobby, but t...  Negative
3      Understand first that this was a party trip wi...  Negative
4      A group of 12 girls and I came here for a Bach...  Negative
...                                                  ...       ...
16197  I am very glad that I stayed here on my recent...  Negative
16198  We booked this hotel for me and my bestie to s...  Negative
16199  Since this hotel is right across the street fr...  Negative
16200  We stayed at the Hilton New Orleans Riverside ...  Negative
16201  Update time....in town for the Yelpy Awards an...  Negative

[16202 rows x 2 columns]


In [ ]:
df.tail(20)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,DriveThru,review_id,user_id,stars_y,useful,funny,cool,text,date,sentiment
16182,GmXC3CBu172Bkb0c6RnO7w,Restaurant R'evolution,777 Bienville St,New Orleans,LA,70130,29.955444,-90.068147,4.5,1210,...,NaN,cM29N94v_FsbfcSSsFVmBg,al-88kr0SAphNMTjdft47Q,5,2,0,0,"None of our food lasted very long, but I manag...",2018-12-19 18:24:19,Negative
16183,GmXC3CBu172Bkb0c6RnO7w,Restaurant R'evolution,777 Bienville St,New Orleans,LA,70130,29.955444,-90.068147,4.5,1210,...,NaN,L3YqmA-CvS8ZL9ZP3U5bkw,QGW7G1DLqBpBCbn3Ygtigg,4,2,26,0,"This review is embarrassingly belated, as I wa...",2013-07-07 04:43:53,Negative
16184,NaN,Hotel Villa Convento,616 Ursulines Ave,New Orleans,LA,70116,29.960763,-90.061405,4.0,88,...,NaN,4_pg7iO-Hojz-ZyY_R1cVQ,BZYOb1-1nD_a0fMp4oCmtA,4,4,1,2,We came to Hotel Villa Convento for the holida...,2017-06-02 01:00:45,Negative
16185,u-wkhjV7CfvGJVY7hGULjw,Hilton New Orleans Riverside,2 Poydras St,New Orleans,LA,70130,29.947506,-90.063454,3.0,559,...,NaN,4H4EEZb-FgcH4KeM44ZE1Q,wNKXxsuI6tNuPjBdjnxDUw,4,1,0,1,This hotel is great. We arrived super early an...,2018-09-26 04:44:27,Negative
16186,u-wkhjV7CfvGJVY7hGULjw,Hilton New Orleans Riverside,2 Poydras St,New Orleans,LA,70130,29.947506,-90.063454,3.0,559,...,NaN,S8r1NkivRHi4ClVu7BDtTw,EPK4_bIsyVlrlsLH3RClEQ,3,4,0,1,The Hilton Riverside is huge and the layout is...,2013-10-15 17:00:47,Negative
16187,auv_gvtR7fZp9b0mkEbTIA,Bienville House Hotel,320 Decatur St,New Orleans,LA,70130,29.953824,-90.065260,4.0,167,...,NaN,xZnWHGIFaSKUGTSybMKH9Q,oixP_0SFBg5nZY1Z55ry2w,5,4,0,0,I chose this place for 3 main reasons. 1) The ...,2020-01-07 01:05:02,Negative
16188,auv_gvtR7fZp9b0mkEbTIA,Bienville House Hotel,320 Decatur St,New Orleans,LA,70130,29.953824,-90.065260,4.0,167,...,NaN,viDTQ86CSNWH8VQKrJQVlg,isQxsxlCjj6P8GQ_uuFwew,4,3,0,2,"Overall, this was a nice hotel to stay in. Th...",2021-11-04 16:03:29,Negative
16189,auv_gvtR7fZp9b0mkEbTIA,Bienville House Hotel,320 Decatur St,New Orleans,LA,70130,29.953824,-90.065260,4.0,167,...,NaN,YOSDCyj9SW5UYZ7njeue5g,ig4hY5dXHOmY9nHxY8Wqvg,5,1,0,0,This place is amazing. I got a suite and it wa...,2019-03-07 22:11:04,Negative
16190,vfQEOutIMu9uKOPaf_4hdg,Sonesta ES Suites New Orleans,212 Loyola Ave,New Orleans,LA,70112,29.954090,-90.074622,3.0,9,...,NaN,d5WOTsOTXa8r90HLwu_q3Q,CCNhUBv0Br_V_4GYIzP1-g,4,2,0,2,Wow! This place is getting slammed with bad re...,2021-05-26 16:39:35,Negative
16191,SQGjAZF6fQ3GTq4A6W3LBA,Comfort Suites New Orleans East,7051 Bullard Ave,New Orleans,LA,70128,30.048769,-89.962607,4.0,14,...,NaN,BnhqLQaQSyuE0mV8ED5t8g,OGYUvEZ7b8hv_XCtoUrnXg,5,1,0,1,"Where to begin? Clean rooms, friendly engagin...",2020-01-16 01:23:56,Negative


In [ ]:
df.to_excel('review_table_s.xlsx')